# Contact Plan Generator

## Libraries

In [ ]:
%config IPCompleter.greedy=True
%matplotlib notebook

from win32api import GetSystemMetrics
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import os

#Needed to interact with COM
from comtypes.client import CreateObject

# Change the notation of numpy
np.set_printoptions(suppress=True)

pd.options.mode.chained_assignment = None  # default='warn'

## Input parameters

In [ ]:
# Input parameters
ScenarioName      = "Walker-53x64x8x5"                # Scenario name
ScenarioStartTime = "1 Jan 2022 00:00:00.000"         # Start date
ScenarioStopTime  = "2 Jan 2022 00:00:00.000"         # End date
EndPointList_path = './inputs/'+ScenarioName+'.csv'   # Path to EndPoint List csv file
Flag_split = True


# Tessellation Params (Grid MxN)
MLon        = 18
NLat        = 9
DeltaLon    = 360/MLon
DeltaLat    = 180/NLat


# Internal parameters
DateFormat        = '%d %b %Y %H:%M:%S.%f'            # Format of date parameters
SecondElapsed     = dt.datetime.strptime(ScenarioStopTime, DateFormat) - dt.datetime.strptime(ScenarioStartTime, DateFormat)
SecondElapsed     = SecondElapsed.total_seconds()     # Total simulation seconds

DataRate_default  = 115200                            # [bits/seg] Datarate assigned to each link
LightSpeed        = 299792                            # [km/s] Velocidad de la luz

# Folder structure
Scenario_path     = os.path.abspath(os.getcwd()) + '/scenario/' # Path to save STK scenario
if not os.path.exists('./scenario'):
    os.makedirs('./scenario')

ContactPlan_path  = './contactPlan/'+ ScenarioName + ".txt"  # Path to save contactPlan
if not os.path.exists('./contactPlan'):
    os.makedirs('./contactPlan')
    
Traffic_path  = './traffic/'+ ScenarioName + ".ini"          # Path to save traffic file
if not os.path.exists('./traffic'):
    os.makedirs('./traffic')

# Load EndPoint List csv files
EndPointList_df   = pd.read_csv(EndPointList_path, sep=",", decimal=",", index_col="Index", 
                                dtype={'Name':str, 'Type':str, 'ApogeeAltitude':float, 'PerigeeAltitude':float, 
                                       'Inclination':float, 'ArgOfPerigee':float, 'RAAN':float, 'TrueAnomaly':float, 
                                       'CnstrMaxRange':float, 'Lat':float, 'Lon':float, 'H':float, 'CnstrMinElev':float})
EndPointList_df['Name']    = EndPointList_df['Name'].str.replace(' ','_')
EndPointList_df['Pointer'] = object

## Create the scenario in STK

In [ ]:
#Import the STKObjects and STKUtil COM libraries
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.client import gen_dir

#Start STK12 Application and configure visualization
uiApp             = CreateObject("STK11.Application")

# NewScenario
stkRoot  = uiApp.Personality2       # Pointer to the IAgStkObjectRoot interface
stkRoot.NewScenario(ScenarioName)   # Scenario name
scenario = stkRoot.CurrentScenario  # POINTER to IAgStkObject

scenario2 = scenario.QueryInterface(STKObjects.IAgScenario) # POINTER to IAgScenario
scenario2.StartTime = ScenarioStartTime  # Scenario Start time
scenario2.StopTime  = ScenarioStopTime   # Scenario Stop time

# Configure visualization
uiApp.Visible     = True
uiApp.UserControl = True
uiApp.Top         = 0
uiApp.Left        = 0
uiApp.Width       = int(GetSystemMetrics(0))
uiApp.Height      = int(GetSystemMetrics(1)-30)
stkRoot.UnitPreferences.Item('DateFormat').SetCurrentUnit('EpSec') # Set Unit preferences

scenario2.Graphics.AccessLinesVisible = True
scenario2.Graphics.AccessAnimHigh = False
scenario2.Graphics.AccessStatHigh = False

stkRoot.ExecuteCommand('MapDetails * LatLon Lat On {} red dashed 1'.format(DeltaLon))
stkRoot.ExecuteCommand('MapDetails * LatLon Lon On {}'.format(DeltaLon))

#Reset STK to the new start time
stkRoot.Rewind()

## Functions

In [ ]:
def AddSatellite (name, apogeeAltitude, perigeeAltitude, inclination, argOfPerigee, raan, trueAnomaly, cnstrMaxRange, propagator):
    global scenario
    #Insert satellite on 'scenario'
    satellite = scenario.Children.New(STKObjects.eSatellite, name)
    satellite2 = satellite.QueryInterface(STKObjects.IAgSatellite)
    
    #Change some basic display attributes and set 'eAttributesCustom'
    satellite2.Graphics.SetAttributesType(2)  # eAttributesCustom
    satellite2.Graphics.PassData.GroundTrack.SetLeadDataType(0)
    satellite2.Graphics.PassData.GroundTrack.SetTrailDataType(0)
    satelliteCustomGfxAttributes =  satellite2.Graphics.Attributes.QueryInterface(STKObjects.IAgVeGfxAttributesCustom)
    satelliteCustomGfxAttributes.Default.Color = 65535 #Yellow
    satelliteCustomGfxAttributes.Default.IsVisible = True
    satelliteCustomGfxAttributes.Default.Line.Width = STKObjects.e1
    satelliteCustomGfxAttributes.Default.Inherit = False
    satelliteCustomGfxAttributes.Default.MarkerStyle = r'D:\Archivos de programa\AGI\STK 11\STKData\Pixmaps\MarkersWin\m010Satellite.bmp'
    
    #Add constraints
    constraints = satellite2.AccessConstraints.AddConstraint(34)
    aux = constraints.value.QueryInterface(STKObjects.IAgAccessCnstrMinMax)
    aux.EnableMin = False
    aux.EnableMax = True
    aux.Max = cnstrMaxRange
    
    #Select Propagator
    if (propagator == 'J2'):
        satellite2.SetPropagatorType(STKObjects.ePropagatorJ2Perturbation)
        propagator = satellite2.Propagator.QueryInterface(STKObjects.IAgVePropagatorJ2Perturbation)
        keplarian = propagator.InitialState.Representation.ConvertTo(STKUtil.eOrbitStateClassical).QueryInterface(STKObjects.IAgOrbitStateClassical)
        
    elif (propagator == 'TwoBody'):
        satellite2.SetPropagatorType(STKObjects.ePropagatorTwoBody)
        propagator = satellite2.Propagator.QueryInterface(STKObjects.IAgVePropagatorTwoBody)
        keplarian = propagator.InitialState.Representation.ConvertTo(STKUtil.eOrbitStateClassical).QueryInterface(STKObjects.IAgOrbitStateClassical)
        
    else:
        print("Error - Bad propagator")
        return 0
        
    #Set initial state
    # Apogee and Perigee Altitude
    keplarian.SizeShapeType = STKObjects.eSizeShapeAltitude
    keplarian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeAltitude).ApogeeAltitude = apogeeAltitude
    keplarian.SizeShape.QueryInterface(STKObjects.IAgClassicalSizeShapeAltitude).PerigeeAltitude = perigeeAltitude
    
    # Inclinación
    keplarian.Orientation.Inclination = inclination

    # Argument of Perigee
    keplarian.Orientation.ArgOfPerigee = argOfPerigee

    # RAAN
    keplarian.Orientation.AscNodeType = STKObjects.eAscNodeRAAN
    keplarian.Orientation.AscNode.QueryInterface(STKObjects.IAgOrientationAscNodeRAAN).Value = raan

    # True anomaly
    keplarian.LocationType = STKObjects.eLocationTrueAnomaly
    keplarian.Location.QueryInterface(STKObjects.IAgClassicalLocationTrueAnomaly).Value = trueAnomaly

    # Propagate
    propagator.InitialState.Representation.Assign(keplarian)
    propagator.Propagate()
    
    return satellite

    
def AddGrounStation (name, lat, lon, h, cnstrMinElev):
    global scenario
    #Insert GS on 'scenario'
    groudStation = scenario.Children.New(STKObjects.eFacility, name)
    groudStation2 = groudStation.QueryInterface(STKObjects.IAgFacility)
    
    #Change some basic attributes
    groudStation2.Position.AssignGeodetic(lat, lon, 0)  # Latitude, Longitude, Altitude
    groudStation2.UseTerrain = True
    groudStation2.HeightAboveGround = h
    groudStation2.Graphics.Color = 16776960

    #Add constraints
    constraints = groudStation2.AccessConstraints.AddConstraint(14)
    aux = constraints.value.QueryInterface(STKObjects.IAgAccessCnstrMinMax)
    aux.EnableMin = True
    aux.EnableMax = False
    aux.Min = cnstrMinElev
    
    return groudStation



def MergeAddress(x,y,gs):
    xygs = (np.left_shift(x,8) | np.left_shift(y,1) | gs)
    return xygs #0b XXXXXXX YYYYYYY Z

def SplitAddress(xygs):
    x = (xygs & (0b1111111<<8))>>8
    y = (xygs & (0b1111111<<1))>>1
    gs = (xygs & (0b1))>>1
    return x, y, gs

def LLA2Addresses(LLA, isGS=0):
    LLA_y = np.floor((LLA[:,1]+90) /  DeltaLat)
    LLA_x = np.floor((LLA[:,2]+180) / DeltaLon)
    LLA_y = LLA_y.astype(np.int32)
    LLA_x = LLA_x.astype(np.int32)

    AddressVsTime = np.c_[LLA[:,0], MergeAddress(LLA_x, LLA_y, isGS)]    

    # Get only the changes of adresses
    _, indices = np.unique(AddressVsTime[:,1],return_index=True)
    AddressVsTime = AddressVsTime[indices]
    AddressVsTime = AddressVsTime[AddressVsTime[:,0].argsort()]

    return AddressVsTime

def GetAddressesVsTime(obj1, obj2, startTime, stopTime, temporalResolution=10):
    # Get position vs time (obj1 and obj2)
    LLAReport = obj1.DataProviders.GetDataPrvInfoFromPath("LLA State/Fixed").QueryInterface(STKObjects.IAgDataPrvTimeVar).ExecElements(startTime, stopTime, temporalResolution, ["Time","Lat", "Lon"])
    LLAObj1   = np.array([list(x) for x in LLAReport.DataSets.ToArray()], dtype=np.float32)
    
    if (obj2.ClassType == 8): # eFacility
        LLAReport = obj2.Position.QueryPlanetodetic()
        LLAObj2 = np.repeat([[LLAObj1[0][0], LLAReport[0], LLAReport[1]]],len(LLAObj1), axis=0)
        obj2IsGS = True
    else:
        LLAReport = obj2.DataProviders.GetDataPrvInfoFromPath("LLA State/Fixed").QueryInterface(STKObjects.IAgDataPrvTimeVar).ExecElements(startTime, stopTime, temporalResolution, ["Time","Lat", "Lon"])
        LLAObj2   = np.array([list(x) for x in LLAReport.DataSets.ToArray()], dtype=np.float32)
        obj2IsGS = False
    
    AddressesObj1 = LLA2Addresses(LLAObj1, isGS=False)
    AddressesObj2 = LLA2Addresses(LLAObj2, isGS=obj2IsGS)
    

    try:
        # Obtener direcciones previas a "StartTime"
        LLAReport = obj1.DataProviders.GetDataPrvInfoFromPath("LLA State/Fixed").QueryInterface(STKObjects.IAgDataPrvTimeVar).ExecElements(startTime-600, startTime, temporalResolution, ["Time","Lat", "Lon"])
        LLAObj1   = np.array([list(x) for x in LLAReport.DataSets.ToArray()], dtype=np.float32)
        
        if (obj2.ClassType == 8): # eFacility
            LLAReport = obj2.Position.QueryPlanetodetic()
            LLAObj2 = np.repeat([[LLAObj1[0][0], LLAReport[0], LLAReport[1]]],len(LLAObj1), axis=0)
            obj2IsGS = True
        else:
            LLAReport = obj2.DataProviders.GetDataPrvInfoFromPath("LLA State/Fixed").QueryInterface(STKObjects.IAgDataPrvTimeVar).ExecElements(startTime-600, startTime, temporalResolution, ["Time","Lat", "Lon"])
            LLAObj2   = np.array([list(x) for x in LLAReport.DataSets.ToArray()], dtype=np.float32)
            obj2IsGS = False
        
        AddressesObj1_prev = LLA2Addresses(LLAObj1, isGS=False)
        AddressesObj2_prev = LLA2Addresses(LLAObj2, isGS=obj2IsGS)
        AddressesObj1_prev = AddressesObj1_prev[AddressesObj1_prev[:,1].astype(int)!=AddressesObj1[0,1].astype(int)][-1,1]
        AddressesObj2_prev = AddressesObj2_prev[AddressesObj2_prev[:,1].astype(int)!=AddressesObj2[0,1].astype(int)][-1,1]
    except:
        AddressesObj1_prev = 0
        AddressesObj2_prev = 0


    return AddressesObj1, AddressesObj2, AddressesObj1_prev, AddressesObj2_prev


def SplitAccessByAddresses(accessIndex, addressesObj1, addressesObj2, addressesObj1_prev, addressesObj2_prev, accessStopTime):
    # puntos en los cuales alguno de los dos obj cambia su direccion
    changes = np.concatenate((addressesObj1[:,0], addressesObj2[:,0]))
    
    # Elimino duplicados
    _, indices = np.unique(changes,return_index=True)
    changes = changes[indices]
    changes.sort()
    

    accessIntervals_aux = []
    
    if len(changes) > 1:
        #accessIntervals_aux.append((accessIndex, addressVsTime[0][0], addressVsTime[1][0], addressVsTime[0][1], addressVsTime[0][2], 0, 0))
        for i in range(len(changes) - 1):

            start = changes[i]
            end = changes[i+1]
            if (end-start) < 5: # Filtro contactos con duracion menor a 5 segundos
                continue

            allPreviousAddressObj1 = addressesObj1[np.where(addressesObj1[:,0] <= start)]
            allPreviousAddressObj2 = addressesObj2[np.where(addressesObj2[:,0] <= start)]
            
            addressObj1AtStart = int(allPreviousAddressObj1[-1, 1])
            addressObj2AtStart = int(allPreviousAddressObj2[-1, 1])
            
            if len(allPreviousAddressObj1) > 1:
                previousAddressObj1 = int(allPreviousAddressObj1[-2, 1])
            else:
                previousAddressObj1 = int(addressesObj1_prev)
            
            if len(allPreviousAddressObj2) > 1:
                previousAddressObj2 = int(allPreviousAddressObj2[-2, 1])
            else:
                previousAddressObj2 = int(addressesObj2_prev)
            
            accessIntervals_aux.append([accessIndex, start, end, addressObj1AtStart, addressObj2AtStart, previousAddressObj1, previousAddressObj2])
        
        
    start = changes[-1]
    end = accessStopTime
    if (end-start) < 5: # Filtro contactos con duracion menor a 5 segundos
        return accessIntervals_aux

    allPreviousAddressObj1 = addressesObj1[np.where(addressesObj1[:,0] <= start)]
    allPreviousAddressObj2 = addressesObj2[np.where(addressesObj2[:,0] <= start)]
    
    addressObj1AtStart = int(allPreviousAddressObj1[-1, 1])
    addressObj2AtStart = int(allPreviousAddressObj2[-1, 1])
    
    if len(allPreviousAddressObj1) > 1:
        previousAddressObj1 = int(allPreviousAddressObj1[-2, 1])
    else:
        previousAddressObj1 = int(addressesObj1_prev)

    if len(allPreviousAddressObj2) > 1:
        previousAddressObj2 = int(allPreviousAddressObj2[-2, 1])
    else:
        previousAddressObj2 = int(addressesObj2_prev)
    
    accessIntervals_aux.append([accessIndex, start, end, addressObj1AtStart, addressObj2AtStart, previousAddressObj1, previousAddressObj2])    

    return accessIntervals_aux


def GetAccessRanges(access_Obj1ToObj2, accessIntervals, temporalResolution=10):
    accessRanges = []
    
    for accessInterval in accessIntervals:
        aerReport = access_Obj1ToObj2.DataProviders.GetDataPrvInfoFromPath("AER Data/Default").QueryInterface(STKObjects.IAgDataPrvTimeVar).ExecElements(float(accessInterval[1]), float(accessInterval[2]), temporalResolution, ["Range"])
        accessRanges.append(np.mean(aerReport.DataSets.ToArray()))
    
    return accessRanges


# obj1 = sat; obj2 = sat or gs
def GetContacts(obj1, obj2, split=True, ranges=False):
    # Names
    obj1_Name = obj1.Path.split('/')[-1]
    obj2_Name = obj2.Path.split('/')[-1]
    print('From ' + obj1_Name + ' to ' + obj2_Name + ' - ', end='')
        
    # Access from obj1 to obt2
    access_Obj1ToObj2 = obj1.GetAccessToObject(obj2)
    access_Obj1ToObj2.ComputeAccess()
            
    # Get access Interval
    accessReport    = access_Obj1ToObj2.DataProviders.Item("Access Data").QueryInterface(STKObjects.IAgDataPrvInterval)
    accessIntervals = accessReport.ExecElements(scenario2.StartTime, scenario2.Stoptime, ["Access Number","Start Time", "Stop Time"])
    accessIntervals = accessIntervals.DataSets.ToArray()
    
    if (len(accessIntervals) == 0):
        print('No contacts')
        return [], [], [], []
            
    # Split all access intervals when obj change address
    accessIntervals_split = []
    if (split==True):
        for accessInterval in accessIntervals:
            # Get Addresses vs Time
            AddressVsTimeObj1, AddressVsTimeObj2, AddressesObj1_prev, AddressesObj2_prev = GetAddressesVsTime(obj1, obj2, accessInterval[1], accessInterval[2], 10)
            
            # Split the access
            accessIntervals_aux = SplitAccessByAddresses(accessIndex=accessInterval[0], addressesObj1=AddressVsTimeObj1, addressesObj2=AddressVsTimeObj2, addressesObj1_prev=AddressesObj1_prev, addressesObj2_prev=AddressesObj2_prev, accessStopTime=accessInterval[2])
            
            # Append access intervals
            accessIntervals_split += accessIntervals_aux
    

    # Get access Ranges
    if (ranges == True):
        accessRanges = GetAccessRanges(access_Obj1ToObj2, accessIntervals)
        accessRanges_split = GetAccessRanges(access_Obj1ToObj2, accessIntervals_split)
    else:
        accessRanges = list(np.repeat([LightSpeed*0.001],len(accessIntervals)))
        accessRanges_split = list(np.repeat([LightSpeed*0.001],len(accessIntervals_split)))
    
    # Format access data and save in a list
    contacts_Obj1ToObj2 = []
    ranges_Obj1ToObj2   = []
    contacts_Obj1ToObj2_split = []
    ranges_Obj1ToObj2_split   = []
    if (split==True):
        for i in range(len(accessIntervals_split)):
            aux_contact = ['a', 'contact', accessIntervals_split[i][1], accessIntervals_split[i][2], obj1_Name, obj2_Name, DataRate_default, accessIntervals_split[i][3], accessIntervals_split[i][4], accessIntervals_split[i][5], accessIntervals_split[i][6]]
            aux_range   = ['a',  'range',  accessIntervals_split[i][1], accessIntervals_split[i][2], obj1_Name, obj2_Name, accessRanges_split[i],  accessIntervals_split[i][3], accessIntervals_split[i][4], accessIntervals_split[i][5], accessIntervals_split[i][6]]
            contacts_Obj1ToObj2_split.append(aux_contact)
            ranges_Obj1ToObj2_split.append(aux_range) 
    
    for i in range(len(accessIntervals)):
        aux_contact = ['a', 'contact', accessIntervals[i][1], accessIntervals[i][2], obj1_Name, obj2_Name, DataRate_default]
        aux_range   = ['a',  'range',  accessIntervals[i][1], accessIntervals[i][2], obj1_Name, obj2_Name, accessRanges[i]]
        contacts_Obj1ToObj2.append(aux_contact)
        ranges_Obj1ToObj2.append(aux_range)         
    
    print('Contacts')
    return contacts_Obj1ToObj2, ranges_Obj1ToObj2, contacts_Obj1ToObj2_split, ranges_Obj1ToObj2_split


def MakeBidireccional(allContacts, allRanges, Flag_split):
    allContacts_inv = copy.deepcopy(allContacts)
    allRanges_inv   = copy.deepcopy(allRanges)

    for i in range(len(allContacts)):
        allContacts_inv[i][4] = allContacts[i][5]
        allContacts_inv[i][5] = allContacts[i][4]

        allRanges_inv[i][4] = allRanges[i][5]
        allRanges_inv[i][5] = allRanges[i][4]

        if Flag_split:
            allContacts_inv[i][7] = allContacts[i][8]
            allContacts_inv[i][8] = allContacts[i][7]

            allContacts_inv[i][9]  = allContacts[i][10]
            allContacts_inv[i][10] = allContacts[i][9]

            allRanges_inv[i][7] = allRanges[i][8]
            allRanges_inv[i][8] = allRanges[i][7]

            allRanges_inv[i][9]  = allRanges[i][10]
            allRanges_inv[i][10] = allRanges[i][9]

    allContacts.extend(allContacts_inv)
    allRanges.extend(allRanges_inv)
    
    return allContacts, allRanges

## Add EndPoints (Satellites and Ground Stations)

In [ ]:
for index, EP in EndPointList_df.iterrows():
    if (EP['Type']=='SAT'):
        EndPointList_df['Pointer'][index] = AddSatellite(name=EP['Name'], apogeeAltitude=EP['ApogeeAltitude'], 
                                                         perigeeAltitude=EP['PerigeeAltitude'], inclination=EP['Inclination'], 
                                                         argOfPerigee=EP['ArgOfPerigee'], raan=EP['RAAN'], trueAnomaly=EP['TrueAnomaly'], 
                                                         cnstrMaxRange=EP['CnstrMaxRange'], propagator='J2')

    elif (EP['Type']=='GS'):
        EndPointList_df['Pointer'][index] = AddGrounStation(name=EP['Name'], lat=EP['Lat'], lon=EP['Lon'], h=EP['H'], 
                                                            cnstrMinElev=EP['CnstrMinElev'])


## Access

In [ ]:
AllContacts_Sat2Sat = []
AllRanges_Sat2Sat   = []
AllContacts_Sat2Sat_split = []
AllRanges_Sat2Sat_split   = []
combinations = []

# Access from sats to sats
for satelliteA in scenario.Children.GetElements(STKObjects.eSatellite):
    for satelliteB in scenario.Children.GetElements(STKObjects.eSatellite):
        if satelliteA != satelliteB:  
            if (not [satelliteB, satelliteA] in combinations):
                combinations.append([satelliteA, satelliteB])
                contacts_SatAToSatB, ranges_SatAToSatB, contacts_SatAToSatB_split, ranges_SatAToSatB_split = GetContacts(satelliteA, satelliteB, split=Flag_split, ranges=False)
                AllContacts_Sat2Sat += contacts_SatAToSatB
                AllRanges_Sat2Sat   += ranges_SatAToSatB
                AllContacts_Sat2Sat_split += contacts_SatAToSatB_split
                AllRanges_Sat2Sat_split   += ranges_SatAToSatB_split


# Make bidireccional
AllContacts_Sat2Sat, AllRanges_Sat2Sat = MakeBidireccional(AllContacts_Sat2Sat, AllRanges_Sat2Sat, Flag_split=False)
AllContacts_Sat2Sat_split, AllRanges_Sat2Sat_split = MakeBidireccional(AllContacts_Sat2Sat_split, AllRanges_Sat2Sat_split, Flag_split=True)



AllContacts_Sat2GS = []
AllRanges_Sat2GS   = []
AllContacts_Sat2GS_split = []
AllRanges_Sat2GS_split   = []

# Access from sats to gs
for satellite in scenario.Children.GetElements(STKObjects.eSatellite):
    for grounstation in scenario.Children.GetElements(STKObjects.eFacility):
        contacts_Sat2GS, ranges_Sat2GS, contacts_Sat2GS_split, ranges_Sat2GS_split = GetContacts(satellite, grounstation, split=Flag_split, ranges=False)
        AllContacts_Sat2GS += contacts_Sat2GS
        AllRanges_Sat2GS   += ranges_Sat2GS
        AllContacts_Sat2GS_split += contacts_Sat2GS_split
        AllRanges_Sat2GS_split   += ranges_Sat2GS_split

# Make bidireccional
AllContacts_Sat2GS, AllRanges_Sat2GS = MakeBidireccional(AllContacts_Sat2GS, AllRanges_Sat2GS, Flag_split=False)
AllContacts_Sat2GS_split, AllRanges_Sat2GS_split = MakeBidireccional(AllContacts_Sat2GS_split, AllRanges_Sat2GS_split, Flag_split=True)

# Resultado esperado
# a [contact/range] [StartTime(second)] [StopTime(second)] [SatA] [SatB] [Datarate/Range] [SatA_Address] [SatB_Address]

## Generate Contact File

In [ ]:
AllContacts = AllContacts_Sat2Sat + AllContacts_Sat2GS
AllRanges   = AllRanges_Sat2Sat   + AllRanges_Sat2GS
AllContacts_split = AllContacts_Sat2Sat_split + AllContacts_Sat2GS_split
AllRanges_split   = AllRanges_Sat2Sat_split   + AllRanges_Sat2GS_split

if Flag_split:
    AllContacts_split_df = pd.DataFrame(AllContacts_split, columns =['a','Comand','Start','Stop','SatA','SatB','Datarate','AddressA','AddressB','LastAddressA','LastAddressB'])
    AllRanges_split_df   = pd.DataFrame(AllRanges_split, columns =['a','Comand','Start','Stop','SatA','SatB','Range','AddressA','AddressB','LastAddressA','LastAddressB'])
    AllContacts_split_df.Start = AllContacts_split_df.Start.astype(float)
    AllContacts_split_df = AllContacts_split_df.sort_values(by=['Start'])
    AllRanges_split_df.Start   = AllRanges_split_df.Start.astype(float)
    AllRanges_split_df   = AllRanges_split_df.sort_values(by=['Start'])

AllContacts_df = pd.DataFrame(AllContacts, columns =['a','Comand','Start','Stop','SatA','SatB','Datarate'])
AllRanges_df   = pd.DataFrame(AllRanges, columns =['a','Comand','Start','Stop','SatA','SatB','Range'])
AllContacts_df.Start = AllContacts_df.Start.astype(float)
AllContacts_df = AllContacts_df.sort_values(by=['Start'])
AllRanges_df.Start   = AllRanges_df.Start.astype(float)
AllRanges_df   = AllRanges_df.sort_values(by=['Start'])

In [ ]:
Contact_Plan_f = open(ContactPlan_path, "w")
for index, j in AllContacts_df.iterrows():
    Contact_Plan_f.write(str(j['a']) + ' ' + str(j['Comand']) + ' ' + str(round(float(j['Start']),1)) + ' ' + str(round(float(j['Stop']),1)) + ' ' + 
                         str(EndPointList_df.index[EndPointList_df['Name']==j['SatA']].tolist()[0]) + ' ' + str(EndPointList_df.index[EndPointList_df['Name']==j['SatB']].tolist()[0]) + ' ' + 
                         str(round(float(j['Datarate']),0)) + '\n')

for index, j in AllRanges_df.iterrows():
    Contact_Plan_f.write(str(j['a']) + ' ' + str(j['Comand']) + ' ' + str(round(float(j['Start']),1)) + ' ' + str(round(float(j['Stop']),1)) + ' ' + 
                         str(EndPointList_df.index[EndPointList_df['Name']==j['SatA']].tolist()[0]) + ' ' + str(EndPointList_df.index[EndPointList_df['Name']==j['SatB']].tolist()[0]) + ' ' + 
                         str(round(j['Range']/LightSpeed,4)) + '\n')

Contact_Plan_f.close()

if Flag_split:
    Contact_Plan_split_f = open(ContactPlan_path.replace(".txt", "_split.txt"), "w")

    for index, j in AllContacts_split_df.iterrows():
        Contact_Plan_split_f.write(str(j['a']) + ' ' + str(j['Comand']) + ' ' + str(round(float(j['Start']),1)) + ' ' + str(round(float(j['Stop']),1)) + ' ' + 
                         str(EndPointList_df.index[EndPointList_df['Name']==j['SatA']].tolist()[0]) + ' ' + str(EndPointList_df.index[EndPointList_df['Name']==j['SatB']].tolist()[0]) + ' ' + 
                         str(round(float(j['Datarate']),0)) + ' ' + str(j['AddressA']) + ' ' + str(j['AddressB']) + ' ' + str(j['LastAddressA']) + ' ' + str(j['LastAddressB']) + '\n')

    for index, j in AllRanges_split_df.iterrows():
        Contact_Plan_split_f.write(str(j['a']) + ' ' + str(j['Comand']) + ' ' + str(round(float(j['Start']),1)) + ' ' + str(round(float(j['Stop']),1)) + ' ' + 
                         str(EndPointList_df.index[EndPointList_df['Name']==j['SatA']].tolist()[0]) + ' ' + str(EndPointList_df.index[EndPointList_df['Name']==j['SatB']].tolist()[0]) + ' ' + 
                         str(round(j['Range']/LightSpeed,4)) + ' ' + str(j['AddressA']) + ' ' + str(j['AddressB']) + ' ' + str(j['LastAddressA']) + ' ' + str(j['LastAddressB']) + '\n')

    Contact_Plan_split_f.close()
    


## Generate Traffic file

In [ ]:
# Open traffic file
Traffic_f = open(Traffic_path, "w")

# write a Header
Traffic_f.write('[General]\n')


NumberOfSats = len(EndPointList_df[EndPointList_df['Type']=='SAT'])
N = int(2000/NumberOfSats)
GS_Index_Start= EndPointList_df[EndPointList_df['Type']=='GS'].index[0]
GS_Index_End  = EndPointList_df[EndPointList_df['Type']=='GS'].index[-1] + 1

for i in range(NumberOfSats):
    Random_Time = list(np.random.randint(0,SecondElapsed*0.75,N))
    Random_Time.sort()
    Random_Time = str(Random_Time).replace('[','"').replace(']','"')

    Random_dst = list(np.random.randint(GS_Index_Start, GS_Index_End, N))
    Random_dst = str(Random_dst).replace('[','"').replace(']','"')

    Traffic_f.write('dtnsim.node[' + str(i+GS_Index_End) + '].app.enable=true\n')
    Traffic_f.write('dtnsim.node[' + str(i+GS_Index_End) + '].app.bundlesNumber="1' + (N-1)*',1' + '"\n')
    Traffic_f.write('dtnsim.node[' + str(i+GS_Index_End) + '].app.start='+ Random_Time + '\n')
    Traffic_f.write('dtnsim.node[' + str(i+GS_Index_End) + '].app.destinationEid=' + Random_dst + '\n')
    Traffic_f.write('dtnsim.node[' + str(i+GS_Index_End) + '].app.size="1000' + (N-1)*',1000' + '"\n')
    Traffic_f.write('\n')

Traffic_f.close()

## Save STK Scenario

In [ ]:
#stkRoot.ExecuteCommand('Save / * "{}"'.format(Scenario_path))
stkRoot.ExecuteCommand('VO * SnapFrame SetValues Format tif')
stkRoot.ExecuteCommand('VO * SnapFrame ToFile "{}"'.format(Scenario_path+ScenarioName+"_3D.tif"))
stkRoot.ExecuteCommand('MapSnap * ToFile "{}" 1'.format(Scenario_path+ScenarioName+"_2D.tif"))


In [ ]:
uiApp.Quit()

In [ ]:
Scenario_path